# Web Scrapping Compiled Yearly US AQI Data
 - https://aqs.epa.gov/aqsweb/airdata/download_files.html#AQI
 - https://aqs.epa.gov/aqsweb/airdata/download_files.html
 - https://aqs.epa.gov/aqsweb/documents/about_aqs_data.html
 - https://aqs.epa.gov/aqsweb/documents/data_api.html#variables
 
 

In [1]:
import pandas as pd
import numpy as np
import zipfile

# if error, No module named 'bs4', run --conda install beautifulsoup4-- in conda env
from bs4 import BeautifulSoup 

from io import BytesIO
from zipfile import ZipFile
import requests
from urllib.request import urlopen

## Web Scrap Download links

In [2]:
# link to EPA website page for daily AQI values
html_page = requests.get('https://aqs.epa.gov/aqsweb/airdata/download_files.html')
soup = BeautifulSoup(html_page.content, 'html.parser')

# find all a href links available on this page
file_urls = soup.findAll('a', href=lambda href: href and href.startswith("daily_aqi_by_county_"))
links = ['https://aqs.epa.gov/aqsweb/airdata/'+link.get('href') for link in file_urls]
print(links)

['https://aqs.epa.gov/aqsweb/airdata/daily_aqi_by_county_2021.zip', 'https://aqs.epa.gov/aqsweb/airdata/daily_aqi_by_county_2020.zip', 'https://aqs.epa.gov/aqsweb/airdata/daily_aqi_by_county_2019.zip', 'https://aqs.epa.gov/aqsweb/airdata/daily_aqi_by_county_2018.zip', 'https://aqs.epa.gov/aqsweb/airdata/daily_aqi_by_county_2017.zip', 'https://aqs.epa.gov/aqsweb/airdata/daily_aqi_by_county_2016.zip', 'https://aqs.epa.gov/aqsweb/airdata/daily_aqi_by_county_2015.zip', 'https://aqs.epa.gov/aqsweb/airdata/daily_aqi_by_county_2014.zip', 'https://aqs.epa.gov/aqsweb/airdata/daily_aqi_by_county_2013.zip', 'https://aqs.epa.gov/aqsweb/airdata/daily_aqi_by_county_2012.zip', 'https://aqs.epa.gov/aqsweb/airdata/daily_aqi_by_county_2011.zip', 'https://aqs.epa.gov/aqsweb/airdata/daily_aqi_by_county_2010.zip', 'https://aqs.epa.gov/aqsweb/airdata/daily_aqi_by_county_2009.zip', 'https://aqs.epa.gov/aqsweb/airdata/daily_aqi_by_county_2008.zip', 'https://aqs.epa.gov/aqsweb/airdata/daily_aqi_by_county_2007.

## Download, Unzip, and Compile

In [3]:
# initiating a dataframe
aqi_df = pd.DataFrame()

#forloop to download, unzip, and compile df
for link in links:
    
    # open url
    resp = urlopen(link)
    
    # read zipfile
    zipfile = ZipFile(BytesIO(resp.read()))
    
    # get the csv file name
    fname = zipfile.namelist()[0]
    
    # convert to pandas dateframe
    df = pd.read_csv(zipfile.open(fname), dtype=object)
    
    # close zipfile we don't need
    zipfile.close()
    
    # concatenate dataframes
    aqi_df = pd.concat([aqi_df, df], ignore_index=True, sort=False)
    
    # let me know how much it is completed.
    print(f'{fname} Completed')

daily_aqi_by_county_2021.csv Completed
daily_aqi_by_county_2020.csv Completed
daily_aqi_by_county_2019.csv Completed
daily_aqi_by_county_2018.csv Completed
daily_aqi_by_county_2017.csv Completed
daily_aqi_by_county_2016.csv Completed
daily_aqi_by_county_2015.csv Completed
daily_aqi_by_county_2014.csv Completed
daily_aqi_by_county_2013.csv Completed
daily_aqi_by_county_2012.csv Completed
daily_aqi_by_county_2011.csv Completed
daily_aqi_by_county_2010.csv Completed
daily_aqi_by_county_2009.csv Completed
daily_aqi_by_county_2008.csv Completed
daily_aqi_by_county_2007.csv Completed
daily_aqi_by_county_2006.csv Completed
daily_aqi_by_county_2005.csv Completed
daily_aqi_by_county_2004.csv Completed
daily_aqi_by_county_2003.csv Completed
daily_aqi_by_county_2002.csv Completed
daily_aqi_by_county_2001.csv Completed
daily_aqi_by_county_2000.csv Completed
daily_aqi_by_county_1999.csv Completed
daily_aqi_by_county_1998.csv Completed
daily_aqi_by_county_1997.csv Completed
daily_aqi_by_county_1996.

## Export dataset
 - added to data folder which is in .gitignore

In [4]:
aqi_df.to_csv('data/aqi_df.csv')

# Objective

#### For an analysis of air quality changes in New York City, we plan to look at the AQI data during the COVID-19 lockdowns during March and April of 2020. This will be compared to AQI data during the same months in years prior. 

Start Date: March 1, 2020
<br>
End Date: March 1, 2021

# Read in Dataset

In [3]:
# US daily AQI data from 1980-01-01 to 2021-05-18
aqi=pd.read_csv('data/aqi_df.csv', usecols={'State Name',
                                            'county Name',
                                            'Date',
                                            'AQI',
                                            'Category',
                                            'Defining Parameter',
                                            'Defining Site',
                                            'Number of Sites Reporting'})

# set columns to strings to keep leading zeros
dtype_dic= {'State Code':str,
            'County Code':str,
            'Site Number':str}
dtype = dtype_dic

# AQI monitor locations
monitors=pd.read_csv('data/aqs_monitors.csv', usecols ={'State Code',
                                                        'County Code',
                                                        'Site Number',
                                                        'Latitude',
                                                        'Longitude',
                                                        'Monitor Type'},
                    dtype=dtype_dic)

In [4]:
# data types of columns
print(monitors.dtypes)
print("")
print(aqi.dtypes)

State Code       object
County Code      object
Site Number      object
Latitude        float64
Longitude       float64
Monitor Type     object
dtype: object

State Name                   object
county Name                  object
Date                         object
AQI                           int64
Category                     object
Defining Parameter           object
Defining Site                object
Number of Sites Reporting     int64
dtype: object


In [5]:
# combine columns together to get site number
monitors['Defining Site']=(monitors['State Code']+'-'+
                           monitors['County Code']+'-'+
                           monitors['Site Number'])

In [23]:
monitors.head()

,State Code,County Code,Site Number,Latitude,Longitude,Monitor Type,Defining Site
0,01,001,0001,32.437458,-86.472891,OTHER,01-001-0001
1,01,001,0001,32.437458,-86.472891,OTHER,01-001-0001
2,01,001,0002,32.428470,-86.443585,SLAMS,01-001-0002
3,01,001,0002,32.428470,-86.443585,SLAMS,01-001-0002
4,01,001,0003,32.332659,-86.791521,OTHER,01-001-0003


In [24]:
monitors.shape[0]

356777

In [25]:
aqi.shape[0]

11392510

In [26]:
aqi.head()

,State Name,county Name,Date,AQI,Category,Defining Parameter,Defining Site,Number of Sites Reporting
0,Alabama,DeKalb,2021-01-01,30,Good,Ozone,01-049-9991,1
1,Alabama,DeKalb,2021-01-02,27,Good,Ozone,01-049-9991,1
2,Alabama,DeKalb,2021-01-03,34,Good,Ozone,01-049-9991,1
3,Alabama,DeKalb,2021-01-04,36,Good,Ozone,01-049-9991,1
4,Alabama,DeKalb,2021-01-05,31,Good,Ozone,01-049-9991,1


In [6]:
# select specific states to compare

states = ['New York','California','Utah']
aqi_subset=aqi.loc[aqi['State Name'].isin(states)]

In [7]:
# merge datasets
aqi_df = aqi_subset.merge(monitors, on='Defining Site', how='left')

In [19]:
aqi_df.head()

,State Name,county Name,County Code_x,Date,AQI,Category,Defining Parameter,Defining Site,Number of Sites Reporting,State Code,County Code_y,Site Number,Latitude,Longitude,Monitor Type
0,California,Colusa,11,2021-01-16,10,Good,PM2.5,06-011-0007,1,06,011,0007,39.021221,-122.281803,TRIBAL
1,California,Colusa,11,2021-01-16,10,Good,PM2.5,06-011-0007,1,06,011,0007,39.021221,-122.281803,TRIBAL
2,California,Colusa,11,2021-01-16,10,Good,PM2.5,06-011-0007,1,06,011,0007,39.021221,-122.281803,TRIBAL
3,California,Colusa,11,2021-01-17,0,Good,PM2.5,06-011-0007,1,06,011,0007,39.021221,-122.281803,TRIBAL
4,California,Colusa,11,2021-01-17,0,Good,PM2.5,06-011-0007,1,06,011,0007,39.021221,-122.281803,TRIBAL


In [8]:
aqi_df['State Name'].unique()

array(['California', 'New York', 'Utah'], dtype=object)

In [31]:
## convert to geodataframe




In [ ]:
aqi.head()

In [ ]:
# format date as datetime
aqi_df['Date']=pd.to_datetime(aqi_df['Date'], format = '%Y-%m-%d')

In [ ]:

plt.figure(figsize=(10, 6), dpi=300)    # create a new figure, set size and resolution (dpi)
plt.scatter(aqi_df['Date'], aqi_df['AQI'], c=aqi_df['Category'])  # add data to the plot

#plt.legend(scatterpoints=1, handles='County Code', frameon=False, labelspacing=1, title='County Code')

plt.title('1980-2021 Daily AQI in California, Ney York, and Utah', fontsize=16)
plt.xlabel('Date', fontsize=14)
plt.ylabel('AQI', fontsize=14)
plt.xlim([np.datetime64('2020-01-01'), np.datetime64('2020-12-31')])
plt.ylim(0,500)